In [4]:
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [5]:
def read_csv(file):
    df = pd.read_csv(file, parse_dates=True, index_col=0)
    return df

## XGBoost for LSTM imputed dataset


In [6]:
X_train = read_csv('output/\CompleteDengueTrainLSTM10_0.csv')
X_test = read_csv('output/\CompleteDengueTestLSTM10_.csv')

y = pd.read_csv('input/\dengue_labels_train.csv', parse_dates=True)

y = y.groupby(['year', 'weekofyear']).sum()
y_train = y.iloc[:734]
y_train = pd.DataFrame(index = X_train.index, data = y_train.total_cases.values)
y_test = y.iloc[734:]
y_test = pd.DataFrame(index = X_test.index, data = y_test.total_cases.values)


xgb_model = xgb.XGBClassifier(objective="reg:squarederror",
                                  eval_metric = 'rmse',
                                  max_depth=3,
                                  n_estimators=150,
                                  learning_rate=0.001
                              )
model = xgb_model.fit(X_train, y_train)

y_pred = model.predict(X_test)

MSE = mse(y_test, y_pred)
RMSE = np.sqrt(MSE)

print("\nRMSE: ", np.round(RMSE, 2))

FileNotFoundError: [Errno 2] File b'output/\\CompleteDengueTrainLSTM10_0.csv' does not exist: b'output/\\CompleteDengueTrainLSTM10_0.csv'

In [ ]:
def run_experiment_multiple_times( n=30):
    result_dict= {'rmse':999999999999, 'best_model':None, 'best_rmse':999999999999}

    for i in range(0,n):
        y = pd.read_csv('input/\dengue_labels_train.csv', parse_dates=True)
        y = y.groupby(['year', 'weekofyear']).sum()

        X_train = read_csv('output/\CompleteDengueTrainLSTM10_'+str(i)+'.csv')
        X_test = read_csv('output/\CompleteDengueTestLSTM10_'+str(i)+'.csv')
        y_train = y.iloc[:734]
        y_train = pd.DataFrame(index = X_train.index, data = y_train.total_cases.values)
        y_test = y.iloc[734:]
        y_test = pd.DataFrame(index = X_test.index, data = y_test.total_cases.values)
        xgb_model = xgb.XGBClassifier(objective="reg:squarederror",
                                          eval_metric = 'rmse',
                                          max_depth=3,
                                          n_estimators=150,
                                          learning_rate=0.001
                                      )
        model = xgb_model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        MSE = mse(y_test, y_pred)
        RMSE = np.sqrt(MSE)
        
        print('Round', i, "rmse", RMSE)
        result_dict['rmse'] += RMSE
        if result_dict['best_rmse'] > RMSE:
            result_dict['best_rmse'] = RMSE
            result_dict['best_model'] = model

    result_dict['rmse'] = result_dict['rmse']/n
    return result_dict     

In [ ]:
result_dict = run_experiment_multiple_times(n=30)

In [ ]:
result_dict